In [6]:
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV

from sign.CONST import AMOUNT_OF_KEYFRAMES

from sign.training.load_data.dynamic_loader import DynamicLoader

CSV_PATH = 'youtube.csv'
NUM_HANDS = 2

dyna_lod = DynamicLoader(AMOUNT_OF_KEYFRAMES)
training_data = dyna_lod.prepare_training_data(CSV_PATH)
_xs,ys,_xs_test,ys_test = training_data.spliterino(random_state=1)
xs,xs_test = np.array(_xs),np.array(_xs_test)

🔥🔥 TrajectoryBuilder is now running in BERTRAM_MODE 🔥🔥


# LogisticRegression

In [6]:

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline

logistic_reg = LogisticRegression(random_state=42,max_iter=10000)
parameters = {'C':range(10,1001,20)}
pre_clf = RandomizedSearchCV(logistic_reg,parameters,n_jobs=-1)
pre_clf.fit(xs,ys)
print(pre_clf.best_estimator_,pre_clf.best_index_,pre_clf.best_params_,pre_clf.best_score_)
print(classification_report(ys_test,pre_clf.predict(np.array(xs_test)),digits=4))

if True:
    from joblib import dump
    dump(pre_clf, 'dynamic_lr.joblib')


# SVC / SVM

In [7]:

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
scaler = StandardScaler(copy=True).fit_transform(xs) # remember it might be best to do this for the training data and never for the test set 😈
parameters = {'C':[float(x) for x in range(100,501,20)],"gamma":[x * 0.1 for x in range(30,51,1)]}
pre_clf = RandomizedSearchCV(SVC(),parameters,n_jobs=-1)
pre_clf.fit(xs,ys)
print(pre_clf.best_estimator_,pre_clf.best_index_,pre_clf.best_params_,pre_clf.best_score_)
print(classification_report(ys_test,pre_clf.predict(xs_test),digits=4))

if True:
    from joblib import dump
    dump(pre_clf, 'dynamic_svm.joblib')



KeyboardInterrupt: 

# Random Forest Classifier

In [ ]:

from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
parameters = {"max_features":["sqrt","log2"],"n_estimators":range(100,1001,100)}
pre_clf = RandomizedSearchCV(random_forest,parameters)
pre_clf.fit(np.array(xs),ys)
print(pre_clf.best_estimator_,pre_clf.best_index_,pre_clf.best_params_,pre_clf.best_score_)
print(classification_report(ys_test,pre_clf.predict(xs_test),digits=4))

if True:
    from joblib import dump
    dump(pre_clf, 'dynamic_rf.joblib')


# Voting Classifier

In [13]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

lr = LogisticRegression(C=930.0,max_iter=10_000)
rf = RandomForestClassifier(random_state=42,n_estimators=500,max_features="sqrt")
svc = make_pipeline(StandardScaler(),SVC(random_state=42,probability=True,gamma=0.4,C=400.0)) # remember probability
voting_clf = VotingClassifier( estimators=[ 
                                           ('lr', lr), 
                                           ('rf', rf), 
                                           ('svc', svc)
]) 
parameters = {"voting":["hard","soft"]}
pre_clf = RandomizedSearchCV(voting_clf,parameters,n_jobs=2)
pre_clf.fit(np.array(xs),ys)
print("VotingClassifier")
print(pre_clf.best_estimator_,pre_clf.best_index_,pre_clf.best_params_,pre_clf.best_score_)
print(classification_report(ys_test,pre_clf.predict(xs_test),digits=4))

if True:
    from joblib import dump
    dump(pre_clf, 'dynamic_vc.joblib')


/mnt/repositories/Bsc-Sign-Language-Recon/backend/.venv/lib/python3.11/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


# Bagging Classifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
lr = LogisticRegression(C=930.0,max_iter=10_000)
rf = RandomForestClassifier(random_state=42,n_estimators=500,max_features="sqrt")
svc = make_pipeline(StandardScaler(),SVC(random_state=42,probability=True,gamma=0.4,C=400.0))

bagging_clf = BaggingClassifier(estimator=[ 
                                           ('lr', lr), 
                                           ('rf', rf), 
                                           ('svc', svc)
])

bagging_clf.fit(xs, ys)
parameters = {"voting":["hard","soft"]}
pre_clf = RandomizedSearchCV(bagging_clf,parameters,n_jobs=-1)
pre_clf.fit(np.array(xs),ys)
print("VotingClassifier")
print(pre_clf.best_estimator_,pre_clf.best_index_,pre_clf.best_params_,pre_clf.best_score_)
print(classification_report(ys_test,pre_clf.predict(xs_test),digits=4))

if True:
    from joblib import dump
    dump(pre_clf, 'dynamic_bc.joblib')
